<h2> Unsupervised Pretraining </h2> 

When we do not have much labeled data, but we cannot find a model trained on a similar task. 

If the data you have is unlabelled, you may still be able to perform unsupervised pretraining on it. 

If you can gather plenty of unlabeled training data, you can try to train the layers one by one starting with the lowest layer and going up. 

<h4> Boltzmaan Machine </h4> 

Instead of using a deterministic step function to decide which value to output, these neurons output 1 with some probability, and 0 otherwise.

If we keep a Boltzmaan machine running long enough, the probbility of observing a perticular outcome will only be a function of the connection weights and bias terms, not of the original configuration.

Training a Boltzmann machine means finding the parameters which make the network approximate the training sets probability distribution.

The point where your model no longer depends on its intital state, it has reached thermal equilibrium where there is no thermal gradient to cause any change. This is a generative model.

<h4> Restricted Boltzmaan Machines </h4> 

RBMs are shallow neural networks with only 2 layers and is an unsupervised training method used to find patterns in data by reconstructing the input. It is called restricted because neurons in the same layer are not connected. 

RBMs can be used for both:
- reconstruction of input
- classify instances lacking classification by using the concept that similar neurons will fire in similar inputs. 